In [0]:
dbutils.widgets.text("order", "/Volumes/test_catalog/landing/json_source_volume/order/")
dbutils.widgets.text("customer", "/Volumes/test_catalog/landing/json_source_volume/customer/")
dbutils.widgets.text("status", "/Volumes/test_catalog/landing/json_source_volume/status/")

In [0]:
files = dbutils.widgets.get("order")

spark.sql(
    f"""
    SELECT * FROM json.`{files}`
    """
).show(3)

In [0]:
%sql
CREATE OR REPLACE TABLE test_catalog.bronze.orders AS
SELECT
	*,
	CURRENT_TIMESTAMP AS processing_time,
	_metadata.file_name AS source_file
FROM
	READ_FILES(
    "/Volumes/test_catalog/landing/json_source_volume/order/", format => "json"
  );

In [0]:
%sql
CREATE OR REPLACE TABLE test_catalog.silver.orders AS
SELECT
  order_id,
  timestamp(order_timestamp) AS order_timestamp,
  customer_id,
  notifications
FROM test_catalog.bronze.orders;

In [0]:
%sql
CREATE OR REPLACE VIEW test_catalog.gold.orders_by_date AS
SELECT
  date(order_timestamp) AS order_date,
  count(*) AS total_daily_orders
FROM test_catalog.silver.orders
GROUP BY date(order_timestamp)
;

In [0]:
%sql
SELECT * FROM test_catalog.gold.orders_by_date
LIMIT 5
;